In [26]:
from copy import deepcopy
import os
import sys
import toml

In [30]:
from matplotlib.ticker import PercentFormatter
from scipy.special import rel_entr
from tqdm.notebook import tqdm
import json
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize as SciOpt
import torch

plt.rcParams.update(
    {
        # "text.usetex": True,
        "text.latex.preamble": r"\usepackage{bm}",
        # Enforce default LaTeX font.
        "font.family": "serif",
        "mathtext.fontset": "dejavuserif",
    }
)
pd.options.display.float_format = '{:,.2f}'.format

In [28]:
# fig, ax = plt.subplots(tight_layout=True)
# labels = ['Workload 1', 'Workload 2']
# width = 0.4
# cost_nom = [1, 5]
# cost_rob = [2, 3]
# xtick = np.arange(len(labels))
# xtick = [0, 0.5]
# rects1 = ax.bar(xtick - width / 2, cost_nom, width=width, label='Nominal', color='pink', hatch='.')
# rects1 = ax.bar(xtick, cost_nom, width=width, label='Nominal', color='pink', align='center')
# rects2 = ax.bar(xtick + width / 2, cost_rob, width=width, label='Robust', color='tab:orange', hatch='x')

# ax.set_yticks([])
# ax.set_ylabel('Cost', fontsize=20)
# ax.set_xticks(xtick, labels, fontsize=20)
# ax.legend(fontsize=24)

# fig.tight_layout()
# plt.show()

In [29]:
# fig, ax = plt.subplots(figsize=(100, 5), tight_layout=True)
# labels = ['Workload 1', 'Workload 2']
# width = 0.1
# cost_nom = np.random.rand(200)
# xtick = list(range(len(cost_nom)))
# cmap = plt.cm.tab10
# colors = cmap(np.arange(len(xtick)) % cmap.N)
# rects1 = ax.bar(xtick, cost_nom, align='center', color=colors)
# ax.set_yticks([])
# ax.set_xticks([])
# ax.set_ylabel('Cost', fontsize=20)
# fig.tight_layout()
# plt.show()

In [32]:
import torch
import torch.nn as nn

In [41]:
class Model(nn.Module):
    def __init__(self, num_features=16):
        super().__init__()
        self.num_features = num_features
        
        self.layers = nn.Sequential(
            nn.Linear(num_features, num_features),
            nn.ReLU(),
            nn.Linear(num_features, num_features),
        )
        
    def forward(self, x, temp=1e-3, hard=False) -> torch.Tensor:
        out = self.layers(x)
        out = nn.functional.gumbel_softmax(out, tau=temp, hard=hard)
        
        return out

In [88]:
num_features = 6
model = Model(num_features)

In [89]:
x = torch.rand(num_features)

In [180]:
pred = model(x, temp=1e-9)
pred

tensor([0., 1., 0., 0., 0., 0.], grad_fn=<SoftmaxBackward0>)

In [181]:
pred.sum().item()

1.0